In [1]:
from ipynb.fs.full.Plot_Noro import *

Loading BokehJS ...

In [2]:
import pandas as pd
from datetime import datetime as dt
import json, time, os
from bokeh.resources import CDN
from bokeh.embed import file_html, json_item
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, curdoc
from bokeh.document import Document
from bokeh.resources import INLINE
from bokeh.util.browser import view


# import holoviews as hv
# hv.extension('bokeh')
# from holoviews import opts


from bokeh.models import DatetimeTickFormatter, Legend, NumeralTickFormatter
from bokeh.layouts import widgetbox, row, layout
from bokeh.models import ColumnDataSource, Toggle, BoxAnnotation, CustomJS
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid, Circle, HoverTool, BoxSelectTool,\
    SaveTool, CustomJS, DatetimeAxis, LinearAxis, NumeralTickFormatter, FactorRange, CategoricalColorMapper,\
    Legend
from bokeh.models.widgets.tables import (
    DataTable, TableColumn, IntEditor
)
from bokeh.models.widgets import DataTable, TableColumn, StringFormatter,\
        NumberFormatter, StringEditor, IntEditor, NumberEditor, SelectEditor,\
        CheckboxButtonGroup, CheckboxGroup, MultiSelect, RadioButtonGroup,\
        Select, Slider, Panel, Tabs, TextInput, Paragraph, Div, Button, Dropdown,\
        DateRangeSlider


from bokeh.palettes import Magma9, Category20b, Category20c
from bokeh.transform import factor_cmap

from bokeh.models.glyphs import Circle
from bokeh.models.layouts import Column, Row
from bokeh.layouts import gridplot

### Simple pie chart for all data

#### Figure 5. GI polymerase genotypes per country (>10 submitted sequences)

In [3]:
def get_slider_state(doc, item_name):
    slider = doc.get_model_by_name(item_name)
    start_date = slider.value[0]
    end_date = slider.value[1]
    if isinstance(start_date, int) or isinstance(start_date, float):
        start_date = dt.fromtimestamp(start_date/1000).strftime("%d-%m-%Y")
        end_date = dt.fromtimestamp(end_date/1000).strftime("%d-%m-%Y")
    else:
        start_date = pd.to_datetime(start_date).strftime("%d-%m-%Y")
        end_date = pd.to_datetime(end_date).strftime("%d-%m-%Y")
        
    return dict(start_date=start_date, end_date=end_date, type='slider', name=slider.name)

def get_checkbox_state(doc, item_name):
    checkbox = doc.get_model_by_name(item_name)
    values = [checkbox.labels[a] for a in checkbox.active]
    return dict(values=values, type='checkbox', name='checkbox')

def get_radiobuttongroup_state(doc, item_name):
    radiobuttongroup = doc.get_model_by_name(item_name)
    value = radiobuttongroup.labels[radiobuttongroup.active]
    return dict(values=value, type='radiobuttongroup', name='radiobuttongroup')

class Dash_base():
    """
    
    :param plot_mode: Dashboard or API. The latter is without control items
    """

    # Items to show
    _layout_items = {'controls':{}, 'plot':None, 'api_url':Div()}
    
    # dataframe
    df = pd.DataFrame()
    
    # categories
    dict_categories = {}    
    
    #own curdoc
    #doc = Document()
    doc = curdoc()
    
    # the url for referencing the custom set plot
    api_url_div = ""
    label = "pie"
    
    # Dashboard or API. The latter is without control items
    plot_mode = "dashboard"
    
    # import logging
# logging.basicConfig(filename='/tmp/app.log', filemode='w', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.DEBUG)
# logger = logging.getLogger(__name__)
    
    @property
    def endpoint(self):
        api_url_prefix = os.getenv("NB_URL")
        api_host = os.getenv("NB_HOST")
        #api_host = "https://kooplex-fiek.elte.hu"
        endpoint = os.path.join(api_host, api_url_prefix, "report")
        return endpoint
        
    def __init__(self, f_dataframe, f_categories=""):
        self.df = pd.read_feather(f_dataframe)   
        #logging.debug('Init class Dash')
        
        with open(f_categories, 'r') as f:
            self.dict_categories = json.loads(f.read())

        # del unknown
        self.df = self.df[self.df['ORF1_sub']!='Unknown']

        
    def on_click_plot(self, new):   
        self.set_api_url()  
    
    def set_controls(self):
        controls = {}
        self._layout_items['controls'] = controls
        
    def init_plot(self):
        item_to_be_treplaced = Div(text="Placeholder for the plot")
        self._layout_items['plot'] = item_to_be_treplaced

    def create_api_url(self):
        self._layout_items['api_url'] =  Div(text="<h3>Placeholder</h3>", name='endpoint_%s'%self.label)
    
    def set_api_url(self):
        controls = self._layout_items['controls']
        
        fields = []
        for item_name in controls.keys():
            control = controls[item_name]
            if isinstance(control, DateRangeSlider):
                values =  get_slider_state(self.doc, item_name)
                fields.append("{name}={start_date},{end_date}".format(**values))
            if isinstance(control, CheckboxGroup):
                result =  get_checkbox_state(self.doc, item_name)
                newfield = "{0}={1}".format(item_name, ",".join(result['values']) if result['values'] else 'all')
                fields.append(newfield)
            if isinstance(control, RadioButtonGroup):
                result =  get_radiobuttongroup_state(self.doc, item_name)
                newfield = "{0}={1}".format(item_name, result['values'])
                fields.append(newfield)
        
        api_url = os.path.join(self.endpoint, "%s?"%self.label+"&".join(fields))
        prev_div = self.doc.get_model_by_name('endpoint_%s'%self.label)
        self.api_url_div = Div(text="<h3>%s</h3>"%api_url, name='endpoint_%s'%self.label)
        prev_div.text = "<h3>%s</h3>"%api_url
        return self.api_url_div
        
        
    def set_layout(self):
        self.doc.add_root(Column())
        layout = []
        if 'controls' in self._layout_items.keys():
            layout.extend([self._layout_items['controls'][kc] for kc in self._layout_items['controls'].keys()])
            if self.plot_mode=='api':
                for kc in self._layout_items['controls'].keys():
                    self._layout_items['controls'][kc].visible=False 
        layout.append(self._layout_items['plot'])
        if 'api_url' in self._layout_items.keys():
            layout.append(self._layout_items['api_url'])
            if self.plot_mode=='api':
                self._layout_items['api_url'].visible=False
        self.doc.roots[-1].children.extend(layout)
        self.set_api_url()


    def get_state_from_url(self, field_dict={}, test_url=""):
        if test_url:
            fields = url.split(self.endpoint)[1]
            if fields[0]=="/":
                fields = fields[1:]
            if fields[0]=="?":
                fields = fields[1:]
            field_dict = {f.split("=")[0]:[f.split("=")[1]] for f in fields.split("&")}
        
        controls = self._layout_items['controls']
        for item_name in field_dict.keys():
            control = controls[item_name]
            if isinstance(control, DateRangeSlider):
                control_item = self.doc.get_model_by_name(item_name)
                new_values = field_dict[item_name][0].split(",")
                new_start_date = pd.to_datetime(new_values[0], dayfirst=True)
                new_end_date = pd.to_datetime(new_values[1], dayfirst=True)
                self.set_control_state(control_item, start_date=new_start_date, end_date=new_end_date)

            if isinstance(control, CheckboxGroup):
                control_item = self.doc.get_model_by_name(item_name)
                new_active_labels = field_dict[item_name][0].split(",")
                self.set_control_state(control_item, labels=new_active_labels)
                
            if isinstance(control, RadioButtonGroup):
                control_item = self.doc.get_model_by_name(item_name)
                new_active_value = field_dict[item_name][0]
                self.set_control_state(control_item, value=new_active_value)    
                
    def set_control_state(self, item, **kwargs):
        if isinstance(item, DateRangeSlider):
            item.value = (kwargs['start_date'], kwargs['end_date'])
        if isinstance(item, CheckboxGroup):
            if kwargs['labels'] != ['all']:
                for l in kwargs['labels']:
                    item.active.append(item.labels.index(l))
        if isinstance(item, RadioButtonGroup):
            item.active = kwargs['value']
        
    def show(self):
         lay=self.doc.roots[0]
#         self.doc.remove_root(self.doc.roots[0])
         return show(lay)        
      

In [4]:
class Dash(Dash_base):
    
    def set_controls(self):
        # 
        controls = {}
    
        # checkbox for countries
        c_choices = list(self.df.Country.unique())
        c_choices.sort()
        checkbox_group = CheckboxGroup(labels=c_choices, active=[], inline=True, name='country_checkbox_%s'%self.label)
        controls['country_checkbox_%s'%self.label] = checkbox_group

        # checkbox for genotype
        g_choices = list(self.df['Genus-Genogroup'].unique())
        g_choices.sort()
        radio_group = RadioButtonGroup(labels=g_choices, active=0, name='genotype_choice_%s'%self.label)
        controls['genotype_choice_%s'%self.label] = radio_group
        
        # time slider
        dates = self.dict_categories['dates']
        min_date = "01-01-1990" #self.df['Sample Date'].min()
        max_date = self.df['Sample Date'].max()
        date_range_slider = DateRangeSlider(title="Date Range: ", start=min_date, end=max_date, value=(min_date, max_date), step=1, name='date_slider_%s'%self.label)
        controls['date_slider_%s'%self.label] = date_range_slider
        
        # button to replot
        b_choose = Button(label="Plot", button_type="success", name="gomb_%s"%self.label)
        b_choose.on_click(self.on_click_plot)
        controls['submit_%s'%self.label] = b_choose
    
        #
        self._layout_items['controls'] = controls 
        
    def on_click_plot(self, new):   
        
        # Get the state of control elements
        BC = self.plot_type(self.df)
        
        # Date Slider
        newtime = get_slider_state(self.doc, 'date_slider_%s'%self.label)
        BC.date_filter = [newtime['start_date'], newtime['end_date']]
        #logging.debug('start: %s, end: %s'%(start, end))

        # Country checkbox
        BC.selected_countries = get_checkbox_state(self.doc, 'country_checkbox_%s'%self.label)['values']    
        
        # Genotype checkbox
#         genotype = 'Caliciviridae Norovirus GI'
        ## CAVEAT only one genotype at once
        BC.selected_genotype = get_radiobuttongroup_state(self.doc, 'genotype_choice_%s'%self.label)['values']
        
        # ORF type radio button
        BC.selected_orf_type = 'ORF1_sub'
        
        # ORF subtype checkbox
        orf_subtypes = []

        BC.filter_df()
        pie_plot = BC.create_plot()
        
        e = self.doc.get_model_by_name('plot_div_%s'%self.label)
        e.children=[pie_plot]
        self.set_api_url()
        
    def do_all(self):
        self.label = np.random.randint(10000)
        self.set_controls()
        self.init_plot()
        self.create_api_url()
        self.set_layout()
        self.set_api_url()

In [5]:
class Dash_pie(Dash):
    """
    :param plot_types: country_pie, outbreak_setting_pie_chart
    """
    
    plot_type = country_pie
    plot_mode = 'dashboard'
    selected_genotype = 'Caliciviridae Norovirus GI'
  
    def init_plot(self):
        BC = self.plot_type(self.df)
        BC.filter_df()
        pie_plot = BC.create_plot()
        pie_plot.tags = ['plott']
        pie_plot.name='plot_div_%s'%self.label
        #item_to_be_treplaced = Row(pie_plot, tags=['plott'], name='plot_div_%s'%self.label)
        item_to_be_treplaced = pie_plot
        self._layout_items['plot'] = item_to_be_treplaced
        
    
        

In [6]:
class Dash_bar(Dash):
    """
    :param plot_types: orf_subtype_bar_chart, factor_bar_chart
    """

    plot_type = orf_subtype_bar_chart
    plot_mode = 'dashboard'
       
    def init_plot(self):
        BC = self.plot_type(self.df)
        #BC.selected_genotype = 'Caliciviridae Norovirus GII'
        BC.filter_df()        
        pie_plot = BC.create_plot()
        #pie_plot = create_bar(df=tdf)
        item_to_be_treplaced = Row(pie_plot, tags=['plott'], name='plot_div_%s'%self.label)
        self._layout_items['plot'] = item_to_be_treplaced
        
  

       

In [7]:
# output_notebook()

# while len(curdoc().roots)>0:
#     curdoc().remove_root(curdoc().roots[0])

In [8]:
D1 = Dash_pie('noronet_all_clean.fr', "categories_all.json")
D1.plot_type = country_pie
#D2.plot_type = factor_bar_chart

D1.do_all()

# D2.label = np.random.randint(10000)

# D2.set_controls()
# D2.init_plot()

# D2.create_api_url()
# D2.set_layout('dashboard')
# #D.set_layout('api')
# D2.set_api_url()

# #url="https://kooplex-fiek.elte.hu/notebook/wfct0p-rere-wfct0p/report/?country_checkbox_pie=FRANCE,GERMANY&date_slider=2018-08-17 00:00:00,2019-02-12 00:00:00"
# #D.get_state_from_url({'country_checkbox_pie': ['HUNGARY,GERMANY'], 'date_slider_pie': ['02-01-2018,02-12-2019']})
# #D.get_state_from_url({'country_checkbox_bar': ['all'], 'date_slider_bar': ['17-04-2018,02-12-2019'],'genotype_checkbox_bar': ['Caliciviridae Norovirus GI']})
# D.on_click_plot('t')

#D.show()

/opt/conda/lib/python3.7/site-packages/bokeh/models/mappers.py:147: UserWarning: Palette length does not match number of factors. ['P17', 'Pm', 'P20', 'Pc', 'Pq', 'P11', 'Pb)', 'P15'] will be assigned to `nan_color` gray
  warnings.warn("Palette length does not match number of factors. %s will be assigned to `nan_color` %s" % (extra_factors, self.nan_color))


In [9]:
D0 = Dash_pie('noronet_all_clean.fr', "categories_all.json")
D0.plot_type = outbreak_setting_pie_chart
D0.do_all()

D2 = Dash_bar('noronet_all_clean.fr', "categories_all.json")
D2.plot_type = orf_subtype_bar_chart
D2.do_all()

D3 = Dash_bar('noronet_all_clean.fr', "categories_all.json")
D3.plot_type = factor_bar_chart
D3.do_all()


# D1.label = np.random.randint(10000)
# D1.set_controls()
# D1.init_plot()

# D1.create_api_url()
# D1.set_layout('dashboard')
# D1.set_api_url()

/opt/conda/lib/python3.7/site-packages/bokeh/models/mappers.py:147: UserWarning: Palette length does not match number of factors. ['P17', 'Pm', 'P20', 'Pc', 'Pq', 'P11', 'Pb)', 'P15'] will be assigned to `nan_color` gray
  warnings.warn("Palette length does not match number of factors. %s will be assigned to `nan_color` %s" % (extra_factors, self.nan_color))
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [10]:
tab1 = Panel(child=D0.doc.roots[0],title="By Country Pie chart")
D0.doc.remove_root(D0.doc.roots[0])

tab2 = Panel(child=D1.doc.roots[0],title="Outbreak Settings Pie chart")
D1.doc.remove_root(D1.doc.roots[0])

tab3 = Panel(child=D2.doc.roots[0],title="By ORF type Bar chart")
D2.doc.remove_root(D2.doc.roots[0])

tab4 = Panel(child=D3.doc.roots[0],title="Factor Bar chart")
D3.doc.remove_root(D3.doc.roots[0])

tabs = Tabs(tabs=[ tab1, tab2, tab3, tab4])
curdoc().add_root(tabs)

In [11]:
#pie_plots = [country_pie, outbreak_setting_pie_chart]
#bar_plots = [orf_subtype_bar_chart, factor_bar_chart]


In [12]:
show(tabs)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [6]:
# class Dash_outbreak_setting_pie_chart(Dash):
    
#     plot_type = outbreak_setting_pie_chart
#     selected_genotype = 'Caliciviridae Norovirus GI'
    
#     def set_controls(self):
#         controls = {}
    
#         # checkbox
#         c_choices = list(self.df.Country.unique())
#         c_choices.sort()
#         checkbox_group = CheckboxGroup(labels=c_choices, active=[], inline=True, name='country_checkbox_%s'%self.label)
#         controls['country_checkbox_%s'%self.label] = checkbox_group

#         # time slider
#         dates = self.dict_categories['dates']
#         min_date = self.df['Sample Date'].min()
#         max_date = self.df['Sample Date'].max()
#         date_range_slider = DateRangeSlider(title="Date Range: ", start=min_date, end=max_date, value=(min_date, max_date), step=1, name='date_slider_%s'%self.label)
#         controls['date_slider_%s'%self.label] = date_range_slider
        
#         # button to replot
#         b_choose = Button(label="Plot", button_type="success", name="gomb")#, callback=CustomJS.from_py_func(on_change_save))
#         b_choose.on_click(self.on_click_plot)
#         controls['submit_%s'%self.label] = b_choose
    
#         self.layout_items['controls'] = controls
  
#     def init_plot(self):
#         BC = self.plot_type(self.df)
#         BC.filter_df()
#         pie_plot = BC.create_plot()
#         item_to_be_treplaced = Row(pie_plot, tags=['plott'], name='plot_div_%s'%self.label)
#         self.layout_items['plot'] = item_to_be_treplaced
        
#     def on_click_plot(self, new):   
        
#         # Get the state of control elements
#         BC = self.plot_type(self.df)
        
#         # Date Slider
#         slider = self.doc.get_model_by_name('date_slider_%s'%self.label)
#         newtime = get_slider_state(slider)
#         BC.date_filter = [newtime['start_date'], newtime['end_date']]
#         #logging.debug('start: %s, end: %s'%(start, end))

#         # Country checkbox
#         checkbox = self.doc.get_model_by_name('country_checkbox_%s'%self.label)
#         BC.selected_countries = get_checkbox_state(checkbox)['values']    
        
#         # Genotype checkbox
# #         genotype = 'Caliciviridae Norovirus GI'
#         ## CAVEAT only one genotype at once
#         BC.selected_genotype = get_checkbox_state(checkbox)['values'][0]
        
#         # ORF type radio button
#         BC.selected_orf_type = 'ORF1_sub'
        
#         # ORF subtype checkbox
#         orf_subtypes = []

#         BC.filter_df()
#         pie_plot = BC.create_plot()
        
#         e = self.doc.get_model_by_name('plot_div_%s'%self.label)
#         e.children=[pie_plot]
#         endpoint = self.doc.get_model_by_name('endpoint_%s'%self.label)
#         self.set_api_url()
        

In [1]:
# def on_event_slider(attr, old, new):
#     start = dt.utcfromtimestamp(new[0]/1000)
#     end = dt.utcfromtimestamp(new[1]/1000)
#     e = curdoc().get_model_by_name('pie')
#     e.children=[create_country_pie(gi_df[gi_df['Sample Date'].apply(lambda x: True if ((x > start) and (x < end)) else False)], g_type="gii")]
#     time.sleep(.2)
    
#date_range_slider.on_change('value', on_event_slider)Message posted 21.4.20.

In [2]:
# #choices = list(df.Country.unique())
# choices = [ 'GI-%s'%s for s in list(gi_df['ORF1_sub'].unique())]
# choices.extend([ 'GII-%s'%s for s in list(gii_df['ORF1_sub'].unique())])
# choices.sort()
# checkbox_group1 = CheckboxGroup(labels=[t for t in choices if 'GI-' in t], active=[], inline=True)
# checkbox_group2 = CheckboxGroup(labels=[t for t in choices if 'GII-' in t], active=[], inline=True)
# d = Div(text="""Choose type""")
# div_choose = Column(d, Row(Column(checkbox_group1, checkbox_group2)))


# Lay = Column(div_choose, tags=['choice'])
# Lay.children[0].children.append(b_choose)
# show(Lay)

In [3]:
# html={}
# html['target_id'] = "myplot2"
# html['root_id'] = 3245
# html['doc'] = doc.to_json()

In [4]:
# while len(curdoc().roots)>0:
#     curdoc().remove_root(curdoc().roots[0])
    
# D = dash('noronet_clean.fr', "categories.json")
# D.set_controls()
# D.init_plot()

# D.create_api_url()
# #D.set_layout('dashboard')
# D.set_layout('api')
# D.set_api_url()

# #url="https://kooplex-fiek.elte.hu/notebook/wfct0p-rere-wfct0p/report/prr?country_checkbox=HUNGARY,GERMANY&date_slider=01-02-2018,12-02-2019"
# D.get_state_from_url({'country_checkbox': ['FRANCE,GERMANY'], 'date_slider': ['10-01-2018,02-12-2019']})
# D.on_click_plot('t')

# D.show()

### Hierarchic pie chart
https://stackoverflow.com/questions/33019879/hierarchic-pie-donut-chart-from-pandas-dataframe-using-bokeh-or-matplotlib

<img src="http://i.stack.imgur.com/YKbzU.png" width="300" />

### Figure 8 Outbreak settings for GI, GII.2, GII.4, GII.6 and GII.17.

## How to create easily
* choose an ORF1_sub type

### Multiple charts
* Use Multichoice or multiselect
* generate all

In [5]:
# import io
# import random

# from jinja2 import Template

# from bokeh.embed import components
# from bokeh.plotting import figure
# from bokeh.resources import INLINE
# from bokeh.util.browser import view

# ########## BUILD FIGURES ################

# PLOT_OPTIONS = dict(plot_width=800, plot_height=300)
# SCATTER_OPTIONS = dict(size=12, alpha=0.5)

# data = lambda: [random.choice([i for i in range(100)]) for r in range(10)]

# red = figure(sizing_mode='scale_width', tools='pan', **PLOT_OPTIONS)
# red.scatter(data(), data(), color="red", **SCATTER_OPTIONS)

# blue = figure(sizing_mode='fixed', tools='pan', **PLOT_OPTIONS)
# blue.scatter(data(), data(), color="blue", **SCATTER_OPTIONS)

# green = figure(sizing_mode='scale_width', tools='pan', **PLOT_OPTIONS)
# green.scatter(data(), data(), color="green", **SCATTER_OPTIONS)

# ########## RENDER PLOTS ################

# # Define our html template for out plots
# template = Template("""\
# <!DOCTYPE html>
# <html lang="en">
#     <head>
#         <meta charset="utf-8">
#         <title>Responsive plots</title>
#         {{ resources }}
#     </head>
#     <body>
#     <h2>Resize the window to see some plots resizing</h2>
#     <h3>Red - pan tool, scale_width</h3>
#     {{ plot_div.red }}
#     <h3>Green - pan tool, scale_width</h3>
#     {{ plot_div.green }}
#     <h3>Blue - pan tool, fixed/h3>
#     {{ plot_div.blue }}

#     {{ plot_script }}
#     </body>
# </html>
# """)

# resources = INLINE.render()

# script, div = components({'red': red, 'blue': blue, 'green': green})

# html = template.render(resources=resources,
#                        plot_script=script,
#                        plot_div=div)

# filename = 'embed_multiple_responsive.html'

# with io.open(filename, mode='w', encoding='utf-8') as f:
#     f.write(html)

# view(filename)
